# From dbt Schema to GDPR-Compliant Pipeline — in 5 Minutes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/01_quickstart/03_dbt_pii_quality.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic)

## The Scenario

You have a **dbt project** for your `customers` model — schema defined, tests written,
PII columns annotated. Now you need:

- ✅ **Quality enforcement** — run the same checks that live in `schema.yml` at ingestion time
- 🔒 **GDPR compliance** — honour erasure requests and mask PII for dev/test environments  
- 🧪 **Test data** — generate realistic synthetic customers that obey the same contract

Normally these would be three separate tools. With LakeLogic + `from_dbt()`, they all
share the same schema file you already maintain.

---

## What we cover

| Step | What happens |
|---|---|
| **1** | Import `dbt_schema.yml` → LakeLogic contract (inspect what was mapped) |
| **2** | Generate synthetic customers from the dbt schema |
| **3** | Run quality validation — see quarantined rows |
| **4** | GDPR: erase a specific customer (nullify / hash / redact strategies) |
| **5** | GDPR: mask all PII for a dev/test copy |
| **6** | Batch erasure + audit report |
| **7** | CLI reference |


## Setup

In [ ]:
# SETUP: Install lakelogic if not already present
from pathlib import Path
import os
import importlib.util
import sys

# --- Package check (works on Colab, Databricks, local) ---
if importlib.util.find_spec("lakelogic") is None:
    print("📦 Installing lakelogic...")
    !pip install lakelogic
    print("✅ lakelogic installed.")
else:
    print("✅ lakelogic is already installed.")

# --- Colab: clone repo so contract files (YAML, etc.) are available ---
if 'google.colab' in sys.modules:
    repo_dir = Path('/content/LakeLogic')
    if not repo_dir.exists():
        print("📂 Cloning LakeLogic repo for example files...")
        !git clone --quiet https://github.com/lakelogic/LakeLogic.git /content/LakeLogic
    os.chdir(repo_dir / 'examples' / '01_quickstart')
    print(f"📍 Working directory set to: {Path.cwd()}")


def find_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        for candidate in [
            base / name,
            base / "examples",
            base / "lakelogic" / "examples"
        ]:
            if candidate.exists():
                return candidate
    return cwd


# Global Path Resolver
def get_full_path(filename: str) -> str:
    path_obj = find_example_dir(filename)
    if path_obj.is_dir():
        return str((path_obj / filename).resolve())
    return str(path_obj.resolve())

SCHEMA_FILE = get_full_path("dbt_schema.yml")

---

## Step 1 — Import dbt Schema → LakeLogic Contract

`DbtAdapter` reads the dbt YAML and maps:
- `meta: {pii: true}` → `field.pii = True`
- `not_null` → `field.required = True`
- `unique` → dataset-level uniqueness rule (`SELECT COUNT(*) - COUNT(DISTINCT ...) FROM ...`)
- `accepted_values` → `row_rule` IN check
- `expression_is_true` → `row_rule` raw SQL

**No manual YAML editing. No new files to maintain.**

In [ ]:
from lakelogic import DataProcessor, DataGenerator, DbtAdapter
from lakelogic.core.gdpr import forget_subjects, mask_pii_columns, generate_erasure_report

adapter  = DbtAdapter(SCHEMA_FILE)
contract = adapter.model_to_contract("customers")

# ── What got mapped? ────────────────────────────────────────────────
fields     = contract.model.fields if contract.model else []
pii_fields = [f.name for f in fields if f.pii]
req_fields = [f.name for f in fields if f.required]
row_rules  = contract.quality.row_rules    if contract.quality else []
ds_rules   = contract.quality.dataset_rules if contract.quality else []

print(f"Contract    : {contract.dataset}")
print(f"Fields      : {len(fields)}")
print(f"Required    : {req_fields}")
print(f"PII fields  : {pii_fields}  ← auto-detected from meta.pii")
print(f"Row rules   : {len(row_rules)}")
print(f"Dataset rules: {len(ds_rules)}")
print()
for r in row_rules:
    print(f"  row rule  ▸ {r.name}: {r.sql}")
for r in ds_rules:
    print(f"  dset rule ▸ {r.name}: {r.sql}")

### Preview the generated LakeLogic contract YAML

You can also export it to disk — once, if you want to add LakeLogic-specific extras
(streaming config, materialization targets, SLOs) on top of what dbt provides.

In [ ]:
# Preview the YAML that would be written
yaml_text = adapter.to_yaml("customers")
print(yaml_text[:2000])  # first 2000 chars

# To write it → contracts/customers.yaml:
#   adapter.to_yaml("customers", output=EXAMPLE_DIR / "contracts" / "customers.yaml")
#
# Or via CLI:
#   lakelogic import-dbt --schema dbt_schema.yml --model customers --output contracts/ --dry-run

---

## Step 2 — Generate Synthetic Customers

`DataGenerator.from_dbt()` reads the same schema file and generates rows that:
- Respect field types (boolean stays boolean, integer stays integer)
- Honour `accepted_values` — `country` and `tier` are always from the allowed set
- Honour `expression_is_true` — `total_spend` is always ≥ 0 for valid rows
- Inject a controlled ratio of **intentionally invalid rows** to test quarantine

No data file needed. Unblock your pipeline on day one.

In [ ]:
gen = DataGenerator.from_dbt(SCHEMA_FILE, model="customers", seed=42)

# 200 rows total: 90% valid, 10% intentionally bad (to exercise quarantine)
df = gen.generate(rows=200, invalid_ratio=0.10)

print(f"Generated  : {len(df)} rows")
print(f"Schema     : {df.schema}")
print()
display(df.head(6))

In [ ]:
# Verify field rules were injected from dbt tests
fr = gen._build_field_rules()
print(f"country accepted values : {fr.get('country', {}).get('accepted_values')}")
print(f"tier    accepted values : {fr.get('tier', {}).get('accepted_values')}")
print(f"total_spend min         : {fr.get('total_spend', {}).get('min')}")
print()

# Spot-check correct types and valid constrained values
print("Sample PII fields:")
display(df.select(["customer_id","email","full_name","tier","country"]).head(4))

---

## Step 3 — Quality Validation

`DataProcessor.from_dbt()` uses the same schema file and runs all the quality rules
that were mapped from dbt tests — at ingestion time, before the data reaches Silver.

The result splits cleanly into **good** (passes all rules) and **quarantined** (fails any rule).

> ℹ️ With `invalid_ratio=0.10` you should see ~90% good, ~10% quarantined.

In [ ]:
proc = DataProcessor.from_dbt(SCHEMA_FILE, model="customers")

good_df, bad_df = proc.run(df)

n_total = len(df)
n_good  = len(good_df) if good_df is not None else 0
n_bad   = len(bad_df)  if bad_df  is not None else 0

print(f"Input rows      : {n_total}")
print(f"Passed (good)   : {n_good}  ({n_good/n_total:.1%})")
print(f"Quarantined     : {n_bad}   ({n_bad/n_total:.1%})")

# Rules that were active:
print()
print("Rules that were active:")
q = proc.contract.quality
if q:
    for r in (q.row_rules or []):
        print(f"  [row]  {r.name}: {r.sql}")
    for r in (q.dataset_rules or []):
        print(f"  [data] {r.name}: {r.sql}")

In [ ]:
# Peek at why bad rows were quarantined
# _lakelogic_errors holds the list of rule names that each row failed
if bad_df is not None and len(bad_df) > 0:
    print("Sample quarantined rows:")
    diag_cols = [c for c in ["customer_id","email","tier","country","total_spend",
                              "_lakelogic_errors","_lakelogic_categories"] if c in bad_df.columns]
    display(bad_df.select(diag_cols).head(5))
else:
    print("No bad rows — all invalid rows may have been caught by required-field rules only.")

---

## Step 4 — GDPR: Right-to-Erasure

A customer contacts support: *"Please delete all my personal data."*

`forget_subjects()` knows which columns are PII **because the dbt schema said so**
via `meta: {pii: true}`. No extra config. Three strategies available:

| Strategy | What happens to PII values | Referential integrity |
|---|---|---|
| `nullify` | Replaced with `null` | Lost |
| `hash` | SHA-256, deterministic | Preserved — can still JOIN on hashed values |
| `redact` | Replaced with `***REDACTED***` | Lost (human-readable audit trail) |

> **Note**: `forget_subjects` casts `customer_id` to string internally for matching,
> so passing `subject_id` as a string works even though the column is `Int64`.

In [ ]:
import polars as pl

# Pick a real customer_id from the good dataset to erase
subject_id = str(good_df["customer_id"][0])
print(f"Erasing customer_id = {subject_id}")
print()

# Show before
before = good_df.filter(pl.col("customer_id").cast(pl.Utf8) == subject_id)
print("Before erasure:")
display(before.select(["customer_id","email","full_name","phone"]))
print()

In [ ]:
# Strategy 1: NULLIFY — PII fields become null
df_nullified = forget_subjects(
    good_df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=[subject_id],
    erasure_strategy="nullify",
)
after_null = df_nullified.filter(pl.col("customer_id").cast(pl.Utf8) == subject_id)
print("After NULLIFY:")
display(after_null.select(["customer_id","email","full_name","phone"]))

In [ ]:
# Strategy 2: HASH — PII replaced with SHA-256, references intact
df_hashed = forget_subjects(
    good_df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=[subject_id],
    erasure_strategy="hash",
    hash_salt="my-secret-salt-2026",
)
after_hash = df_hashed.filter(pl.col("customer_id").cast(pl.Utf8) == subject_id)
print("After HASH (PII is deterministically hashed — can still JOIN):")
display(after_hash.select(["customer_id","email","full_name","phone"]))

In [ ]:
# Strategy 3: REDACT — human-readable, best for audit logs
df_redacted = forget_subjects(
    good_df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=[subject_id],
    erasure_strategy="redact",
)
after_redact = df_redacted.filter(pl.col("customer_id").cast(pl.Utf8) == subject_id)
print("After REDACT:")
display(after_redact.select(["customer_id","email","full_name","phone"]))

### Generate a compliance-ready audit report

This is what you send to the DPO or attach to a DSAR (Data Subject Access Request) response.

In [ ]:
import json

report = generate_erasure_report(
    contract=proc.contract,
    subject_column="customer_id",
    subject_ids=[subject_id],
    erasure_strategy="nullify",
    affected_rows=len(before),
)

print(json.dumps(report, indent=2, default=str))

---

## Step 5 — GDPR: Mask All PII for Dev / Test

You want to give the data engineering team a copy of production data to test with —
but with all PII stripped. `mask_pii_columns()` blanket-anonymises every field
marked `pii: true` in the contract (which we got from `meta: {pii: true}` in dbt).

In [ ]:
df_masked = mask_pii_columns(
    good_df,
    proc.contract,
    strategy="redact",  # or "nullify" or "hash"
)

print("PII fields in dev copy:")
pii_preview = ["customer_id","email","full_name","phone","date_of_birth"]
display(df_masked.select([c for c in pii_preview if c in df_masked.columns]).head(5))
print()
print("Non-PII fields are untouched:")
display(df_masked.select(["customer_id","tier","country","total_spend","is_active"]).head(5))

---

## Step 6 — Batch Erasure + Audit Report

In practice, erasure requests come in batches. One call handles them all.

In [ ]:
# Simulate a batch of 5 erasure requests received this week
erasure_ids = [str(good_df["customer_id"][i]) for i in range(5)]
print(f"Batch erasure for {len(erasure_ids)} subjects: {erasure_ids}")

df_batch_erased = forget_subjects(
    good_df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=erasure_ids,
    erasure_strategy="hash",
    hash_salt="weekly-erasure-salt",
)

# Verify erased rows are hashed
erased_sample = df_batch_erased.filter(
    pl.col("customer_id").cast(pl.Utf8).is_in(erasure_ids)
)
print(f"\nErased rows: {len(erased_sample)}")
display(erased_sample.select(["customer_id","email","full_name"]).head(5))

# DPO audit report for the batch
batch_report = generate_erasure_report(
    contract=proc.contract,
    subject_column="customer_id",
    subject_ids=erasure_ids,
    erasure_strategy="hash",
    affected_rows=len(erased_sample),
)
print()
import json
print(json.dumps(batch_report, indent=2, default=str))

---

## Step 7 — CLI Reference

Everything above is also available from the terminal — no notebook required.

In [ ]:
schema_path = str(SCHEMA_FILE)

print("# ── Import dbt schema → LakeLogic contract ─────────────────────────")
print(f"lakelogic import-dbt --schema {schema_path} --model customers --output contracts/")
print()
print("# ── Dry-run preview (no files written) ──────────────────────────────")
print(f"lakelogic import-dbt --schema {schema_path} --model customers --dry-run")
print()
print("# ── Import every model in the schema file ────────────────────────────")
print(f"lakelogic import-dbt --schema {schema_path} --output contracts/")
print()
print("# ── Run validation against a data file ──────────────────────────────")
print(f"lakelogic run --contract contracts/customers.yaml --source data/customers.csv")
print()
print("# ── Generate 500 synthetic test rows ─────────────────────────────────")
print(f"lakelogic generate --contract contracts/customers.yaml --rows 500 --invalid-ratio 0.1 --output data/test_customers.parquet")

---

## Summary

```
dbt_schema.yml          ← the ONLY schema file you maintain
         │
         ├─ DbtAdapter()              → DataContract (PII, required, rules all mapped)
         │
         ├─ DataGenerator.from_dbt()  → synthetic test data  (Step 2)
         ├─ DataProcessor.from_dbt()  → quality validation    (Step 3)
         ├─ forget_subjects()         → GDPR right-to-erasure (Step 4)
         ├─ mask_pii_columns()        → dev/test PII masking  (Step 5)
         └─ generate_erasure_report() → DPO audit trail       (Steps 4 & 6)
```

**One schema file → zero duplication → full compliance.**

### Next steps

- **`02_core_patterns`** — Medallion architecture and reference joins
- **`03_advanced_workflows/polars_streaming`** — Process 50 GB files without OOM
- **`03_advanced_workflows/gdpr_compliance`** — Full GDPR demo with audit reports
- **`03_advanced_workflows/shared_governance_scale`** — 26 contracts in parallel with shared templates
